### [Evaluating RAG Pipelines](https://haystack.deepset.ai/tutorials/35_evaluating_rag_pipelines)

In [1]:
#! Solo es para haystack sepa que tutorial se esta ejecutando
from haystack.telemetry import tutorial_running
tutorial_running(35)

### 1. Crear el pipeline RAG para evaluar


In [2]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("vblagoje/PubMedQA_instruction", split="train")
dataset = dataset.select(range(1000))

all_documents = [Document(content=doc["context"]) for doc in dataset]
all_questions = [doc["instruction"] for doc in dataset]
all_ground_truth_answers = [doc["response"] for doc in dataset]

README.md:   0%|          | 0.00/498 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/274M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/986k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/272458 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Crear el pipeline para indexar datos de forma sensilla

from typing import List
from haystack import Pipeline
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.document_stores.types import DuplicatePolicy

document_store = InMemoryDocumentStore()

document_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
document_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.SKIP)

indexing = Pipeline()
indexing.add_component(instance=document_embedder, name="document_embedder")
indexing.add_component(instance=document_writer, name="document_writer")

indexing.connect("document_embedder.documents", "document_writer.documents")

indexing.run({"document_embedder": {"documents": all_documents}})

/home/nahumfg/GithubProjects/TesisMaestria/TesisHaystack/venv_haystack_ai/lib/python3.11/site-packages/sentence_transformers/SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

{'document_writer': {'documents_written': 1000}}

In [5]:
# Ahora con los datos listos, crear un RAG simple
import os
from getpass import getpass
from haystack.components.builders import AnswerBuilder, PromptBuilder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.generators import OpenAIGenerator
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

template = """
        You have to answer the following question based on the given context information only.

        Context:
        {% for document in documents %}
            {{ document.content }}
        {% endfor %}

        Question: {{question}}
        Answer:
        """

rag_pipeline = Pipeline()
rag_pipeline.add_component(
    "query_embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
)
rag_pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store, top_k=3))
rag_pipeline.add_component("prompt_builder", PromptBuilder(template=template))
rag_pipeline.add_component("generator", OpenAIGenerator(model="gpt-4o-mini"))
rag_pipeline.add_component("answer_builder", AnswerBuilder())

rag_pipeline.connect("query_embedder", "retriever.query_embedding")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "generator")
rag_pipeline.connect("generator.replies", "answer_builder.replies")
rag_pipeline.connect("generator.meta", "answer_builder.meta")
rag_pipeline.connect("retriever", "answer_builder.documents")

🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - generator: OpenAIGenerator
  - answer_builder: AnswerBuilder
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - retriever.documents -> answer_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)
  - generator.replies -> answer_builder.replies (List[str])
  - generator.meta -> answer_builder.meta (List[Dict[str, Any]])

In [6]:
question = "Do high levels of procalcitonin in the early phase after pediatric liver transplantation indicate poor postoperative outcome?"

response = rag_pipeline.run(
    {
        "query_embedder": {"text": question},
        "prompt_builder": {"question": question},
        "answer_builder": {"query": question},
    }
)
print(response["answer_builder"]["answers"][0].data)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Yes, high levels of procalcitonin (PCT) in the early phase after pediatric liver transplantation are associated with poor postoperative outcomes. Patients with elevated PCT levels on postoperative day 2 had higher International Normalized Ratio values on postoperative day 5, experienced more primary graft non-function, had longer stays in the pediatric intensive care unit, and required more time on mechanical ventilation.


### 2. Evaluar el pipeline

In [7]:
import random

questions, ground_truth_answers, ground_truth_docs = zip(
    *random.sample(list(zip(all_questions, all_ground_truth_answers, all_documents)), 25)
)


In [ ]:
# A continuación, ejecutemos nuestro pipeline y asegurémonos de realizar un seguimiento de lo que nuestro pipeline devuelve como respuestas y qué documentos recupera:
rag_answers = []
retrieved_docs = []

for question in list(questions):
    response = rag_pipeline.run(
        {
            "query_embedder": {"text": question},
            "prompt_builder": {"question": question},
            "answer_builder": {"query": question},
        }
    )
    print(f"Question: {question}")
    print("Answer from pipeline:")
    print(response["answer_builder"]["answers"][0].data)
    print("\n-----------------------------------\n")

    rag_answers.append(response["answer_builder"]["answers"][0].data)
    retrieved_docs.append(response["answer_builder"]["answers"][0].documents)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Does ascorbic Acid ameliorate nicotine exposure induced impaired spatial memory performances in rats?
Answer from pipeline:
Yes, ascorbic acid ameliorates nicotine exposure induced impaired spatial memory performances in rats.

-----------------------------------



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Do [ Recovery of maxillary tooth sensibility after Le Fort I osteotomy ]?
Answer from pipeline:
Yes, the recovery of maxillary tooth sensitivity after Le Fort I osteotomy was analyzed in the study. It was observed that sensitivity recovery was faster in younger patients (under 35 years of age) and for upper middle and superior alveolar nerves. Sensitivity percentages were recorded at different intervals: 91.9% were sensitive pre-surgery (D-1), which dropped to 12.7% at D2, and gradually increased to 61.8% by M6.

-----------------------------------



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Question: Does first registry result from the newly approved ACURATE TA™ TAVI system†?
Answer from pipeline:
The first registry results from the newly approved ACURATE TA™ TAVI system indicate a procedural success rate of 98% with 245 out of 250 patients experiencing successful implantation. The 30-day mortality rate was 6.8%, and the post-implant echocardiography showed a relevant paravalvular leak (moderate 2+) in 2.3% of patients. Additionally, the mean transvalvular gradient post-implantation was 12.4 ± 5.8 mmHg, and 85% of patients followed up presented in NYHA class I/II.

-----------------------------------



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Does qS-21 enhance the early antibody response to oil adjuvant foot-and-mouth disease vaccine in cattle?
Answer from pipeline:
Yes, the addition of QS-21 to the commercially available foot-and-mouth disease water-in-oil-in-water emulsion vaccine showed a significant early antibody increase in the QS-21 group after vaccination in cattle.

-----------------------------------



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Are lobar microbleeds associated with a decline in executive functioning in older adults?
Answer from pipeline:
Yes, lobar microbleeds are associated with a decline in executive functioning in older adults. The study found that participants with 2 or more lobar microbleeds had worse executive functioning at the visit closest to the MRI scan and exhibited a faster decline in executive function over time compared to those with fewer than 2 lobar microbleeds.

-----------------------------------



In [9]:
# Si bien cada evaluador es un componente que se puede ejecutar de forma individual en Haystack, también se pueden agregar a un pipeline. De esta manera, podemos construir un pipeline eval_pipelineque incluya todos los evaluadores para las métricas que queremos evaluar.

from haystack.components.evaluators.document_mrr import DocumentMRREvaluator
from haystack.components.evaluators.faithfulness import FaithfulnessEvaluator
from haystack.components.evaluators.sas_evaluator import SASEvaluator

eval_pipeline = Pipeline()
eval_pipeline.add_component("doc_mrr_evaluator", DocumentMRREvaluator())
eval_pipeline.add_component("faithfulness", FaithfulnessEvaluator())
eval_pipeline.add_component("sas_evaluator", SASEvaluator(model="sentence-transformers/all-MiniLM-L6-v2"))

results = eval_pipeline.run(
    {
        "doc_mrr_evaluator": {
            "ground_truth_documents": list([d] for d in ground_truth_docs),
            "retrieved_documents": retrieved_docs,
        },
        "faithfulness": {
            "questions": list(questions),
            "contexts": list([d.content] for d in ground_truth_docs),
            "predicted_answers": rag_answers,
        },
        "sas_evaluator": {"predicted_answers": rag_answers, "ground_truth_answers": list(ground_truth_answers)},
    }
)


/home/nahumfg/GithubProjects/TesisMaestria/TesisHaystack/venv_haystack_ai/lib/python3.11/site-packages/transformers/models/auto/configuration_auto.py:1006: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/nahumfg/GithubProjects/TesisMaestria/TesisHaystack/venv_haystack_ai/lib/python3.11/site-packages/sentence_transformers/SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(
100%|██████████| 25/25 [00:53<00:00,  2.14s/it]


In [10]:
# Una vez que hayamos ejecutado nuestro proceso de evaluación, también podemos crear un informe de evaluación completo. Haystac proporciona un EvaluationRunResultque podemos usar para mostrar un score_report👇

from haystack.evaluation.eval_run_result import EvaluationRunResult

inputs = {
    "question": list(questions),
    "contexts": list([d.content] for d in ground_truth_docs),
    "answer": list(ground_truth_answers),
    "predicted_answer": rag_answers,
}

evaluation_result = EvaluationRunResult(run_name="pubmed_rag_pipeline", inputs=inputs, results=results)
evaluation_result.score_report()

,metrics,score
0,doc_mrr_evaluator,0.960000
1,faithfulness,0.976667
2,sas_evaluator,0.691143


In [11]:
# Convertir el informe en un Pandas DataFrame
results_df = evaluation_result.to_pandas()
results_df

,question,contexts,answer,predicted_answer,doc_mrr_evaluator,faithfulness,sas_evaluator
0,Does ascorbic Acid ameliorate nicotine exposur...,[The long lasting behavioural and cognitive im...,Ascorbic acid supplementation was found to be ...,"Yes, ascorbic acid ameliorates nicotine exposu...",1.0,1.000000,0.761964
1,Do [ Recovery of maxillary tooth sensibility a...,"[Upper alveolar nerves, when injured during Le...",Among the teeth that were sensitive before sur...,"Yes, the recovery of maxillary tooth sensitivi...",1.0,1.000000,0.775067
2,Do apitherapy products enhance the recovery of...,[Our objective was to identify the antioxidant...,"Our results suggest that honey, propolis, poll...","Yes, apitherapy products, such as honeybee pro...",0.5,1.000000,0.829854
3,Are glutathione s-transferase m1 and t1 gene p...,[The aim of this study was to investigate whet...,This study shows no association between GST ge...,"No, glutathione S-transferase M1 (GSTM1) and T...",1.0,1.000000,0.599473
4,Is bone marrow from blotchy mice dispensable t...,[The blotchy mouse caused by mutations of ATP7...,"Overall, our study indicated that the effect o...","Yes, bone marrow from blotchy mice is dispensa...",1.0,1.000000,0.690724
5,Do the effects of ifenprodil on the activity o...,"[According to reports in the literature, more ...",The concomitant administration of certain comm...,Ifenprodil potentiated the antidepressant-like...,1.0,1.000000,0.537249
6,Does concern for Lost Income Following Donatio...,[Some living kidney donors report lost income ...,Findings from the current study underscore how...,"Yes, concern for lost income following donatio...",1.0,1.000000,0.701326
7,Is pseudomonas aeruginosa in CF and non-CF hom...,[For patients with cystic fibrosis (CF) Pseudo...,These findings implicate drains as important p...,"Yes, Pseudomonas aeruginosa is found predomina...",1.0,1.000000,0.662766
8,Do obese patients with idiopathic pulmonary fi...,[Obese patients with idiopathic pulmonary fibr...,Our results suggest that obese patients who re...,"Yes, obese patients with idiopathic pulmonary ...",1.0,1.000000,0.576158
9,Does introduction of an 8-aminooctanoic acid l...,[The purpose of this study was to examine the ...,High melanoma uptake and fast urinary clearanc...,"Yes, the introduction of an 8-aminooctanoic ac...",1.0,1.000000,0.573848


In [12]:
import pandas as pd

top_3 = results_df.nlargest(3, "sas_evaluator")
bottom_3 = results_df.nsmallest(3, "sas_evaluator")
pd.concat([top_3, bottom_3])


,question,contexts,answer,predicted_answer,doc_mrr_evaluator,faithfulness,sas_evaluator
20,Do systematic Reviews Published in Emergency M...,[Publication bias compromises the validity of ...,Systematic reviews published in emergency medi...,"No, systematic reviews published in emergency ...",1.0,1.000000,0.935371
19,Does glioma-derived versican promote tumor exp...,[Accumulation and infiltration of microglia/br...,Our results show that versican released from g...,"Yes, glioma-derived versican promotes tumor ex...",1.0,1.000000,0.913509
2,Do apitherapy products enhance the recovery of...,[Our objective was to identify the antioxidant...,"Our results suggest that honey, propolis, poll...","Yes, apitherapy products, such as honeybee pro...",0.5,1.000000,0.829854
18,Does surveillance provide insight into epidemi...,[Longer-term tuberculosis (TB) drug resistance...,There has been a significant reduction in bact...,"Yes, surveillance provides insight into the ep...",0.5,1.000000,0.384376
5,Do the effects of ifenprodil on the activity o...,"[According to reports in the literature, more ...",The concomitant administration of certain comm...,Ifenprodil potentiated the antidepressant-like...,1.0,1.000000,0.537249
21,Is p63 more sensitive and specific than 34βE12...,[Prostate cancer is the world's leading cause ...,Basal cell markers can help to distinguish pro...,The context does not provide a direct comparis...,1.0,0.666667,0.544149
